# Proyecto Final — Inferencia sobre Datos Nuevos

En este notebook aplicamos los modelos entrenados sobre datos nuevos para evaluar su capacidad de generalización. El flujo garantiza coherencia con el entrenamiento mediante la reutilización de pipelines y metadatos, permitiendo inferencia reproducible sobre cualquier dataset con la estructura esperada.


## Configuración y dependencias

Definimos las rutas a los artefactos entrenados y al dataset de inferencia. Para aplicar el modelo sobre datos nuevos, modificamos `DATA_FILE` apuntando al archivo correspondiente manteniendo la misma estructura de columnas del entrenamiento.


In [204]:
from typing import Literal, Tuple
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
import warnings
warnings.filterwarnings('ignore')

# Configuración de la tarea
TAREA: Literal["clasificacion", "regresion"] = "clasificacion"

# Rutas - Modificar DATA_FILE para aplicar sobre datos nuevos
DATA_FILE = Path("data/proy_supermercado_test.csv")

MODEL_FILE = Path("models/pipeline_clasificacion_sin_leakage.pkl") if TAREA == "clasificacion" else Path("models/pipeline_regresion_sin_leakage.pkl")

METADATA_FILE = Path("models/pipeline_metadata.pkl")
TARGET_COL = "respuesta" if TAREA == "clasificacion" else "gasto_total"

## Carga del dataset de inferencia

Cargamos el dataset sobre el cual aplicaremos el modelo. Este debe tener la misma estructura de columnas que el dataset de entrenamiento, aunque pueden diferir en el número de observaciones.


In [205]:
assert DATA_FILE.exists(), f"Dataset no encontrado: {DATA_FILE}"

df_raw = pd.read_csv(DATA_FILE)

print(f"Dataset cargado: {df_raw.shape[0]} observaciones, {df_raw.shape[1]} variables")
print(f"Variable objetivo disponible: {TARGET_COL in df_raw.columns}")
df_raw.head()


Dataset cargado: 351 observaciones, 38 variables
Variable objetivo disponible: True


,id,nombre,apellidos,anio_nacimiento,direccion,telefono1,telefono2,email,dni,tarjeta_credito_asociada,...,acepta_cmp3,acepta_cmp4,acepta_cmp5,acepta_cmp1,acepta_cmp2,reclama,coste_contacto,ingresos_contacto,respuesta,usuario_alta_datos
0,1,8e7ebfed1f1543fa81c81459dc89a4d9,651c1322dcc54f9196b6c9da25e6f979,1982.0,70b26c2715614ba19c8bf6f59c4680b4,c468424689a74a9d81cef105c57a5675,dc2f259a84c047b59269b4e71d4b0e26,7e810d39857942099e1a06f340ad8a09,0ca7475698ce48b39f5793790173e712,36751be5d859455fb7aa905eac7f0806,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,0.0,admin
1,24,9652fce167de476cbe7eb6006f4e10fe,c2ccf8175b73472dbffadd4d813fb411,1964.0,4d19f3d14360445e8e6093c56a0abd56,17ba13b5501a43bcae0bc4181c3eda90,3a4224bd3d1b4171b0f5df92ef73aff7,ebf1d2b586a44189a37413b078603efa,131da2e948634187a9baa76769f2621f,fe2ab2a89ee746f0985034939a4592c9,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,0.0,juan.perez
2,36,87c36b0440ce40088cf3a8730e72ca90,7153523e036d4dc4b167a2c64c3bad8d,NaN,b3636305bbdc4b1f827800f179487d9b,9c8683cfdc234741a739f7b23d8c743d,46b3057e7db84998acfeda811c3b6b86,97542b779f144d9fb054ae50ada28a24,a4c128bbc8d24a9680595d9b33ef0357,a0b00686b4174c79aceca58ad47a7e40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,juan.perez
3,40,432bc28e21cf4a4492efae9b8f2dd0c8,6a2b00e861794a46992593a151197fff,1963.0,4e2d9d499b5e46918c38e628f520027c,80d4f81e370e49a18362f4a7b48d9ad8,233bbe4abd8241a5bba8648694120f11,6085bc226bc0488da8661ac71dd1ca4e,2a2e3bf0f1494bf69234a153dfb77273,9f7d46da0c6f41d8b15e21677f6158d8,...,no,no,no,no,no,0.0,3.0,11.0,0.0,admin
4,41,1bfa4d7aaca84a8d96785acb4412d9c1,09cfe74d95fc46a9908b114db7988141,1949.0,7f63c6885f4f45b58ca693e5a1dba298,abe303766efa422e9b7f59a6c7bd50d1,7a81508356334340bcdae67a03d28c57,2a95b42814964d438911905ce62ecc1c,59f4e9f9e01c43a184023868a6cd10e4,182117d9dba24c448e1ea24fa8f59f4b,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,0.0,juan.perez


## Carga de artefactos de entrenamiento

Cargamos el pipeline entrenado y los metadatos que contienen información sobre las features esperadas, transformaciones aplicadas y configuración del modelo.


In [206]:
assert METADATA_FILE.exists(), f"Metadatos no encontrados: {METADATA_FILE}"
assert MODEL_FILE.exists(), f"Modelo no encontrado: {MODEL_FILE}"

metadata = joblib.load(METADATA_FILE)
model = joblib.load(MODEL_FILE)

print(f"Pipeline cargado: {type(model).__name__}")
if TAREA == "clasificacion":
    print(f"Umbral de clasificación: {metadata.get('clf_threshold', 0.5):.4f}")


Pipeline cargado: Pipeline
Umbral de clasificación: 0.3500


## Funciones auxiliares

Definimos funciones para validar y preparar las features del dataset de inferencia, garantizando alineación exacta con el espacio de features utilizado durante el entrenamiento.


In [207]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)

def validar_columnas(work: pd.DataFrame, expected: list[str], contexto: str, allow_extras: bool = False) -> pd.DataFrame:
    faltantes = [c for c in expected if c not in work.columns]
    extras = [c for c in work.columns if c not in expected]

    if faltantes:
        msg = (
            f"Desalineación de columnas en {contexto}. "
            f"Faltantes: {faltantes or 'ninguna'} | Extras: {extras or 'ninguna'}"
        )
        raise ValueError(msg)

    if extras and not allow_extras:
        msg = (
            f"Desalineación de columnas en {contexto}. "
            f"Faltantes: {faltantes or 'ninguna'} | Extras: {extras or 'ninguna'}"
        )
        raise ValueError(msg)

    if extras:
        print(f"Descartamos columnas extra en {contexto}: {extras}")
        work = work.drop(columns=extras)

    return work[expected]


def _first_existing(df: pd.DataFrame, candidates: list[str]) -> str | None:
    for c in candidates:
        if c in df.columns:
            return c
    return None


def preprocesamiento_deterministico_01b(df: pd.DataFrame, target_col: str, meta: dict | None = None) -> pd.DataFrame:
    """
    Replica lo determinístico de 01B y, si faltan columnas esperadas, intenta derivarlas
    desde columnas base típicas (sin fits).
    """
    work = df.copy()

    # fechas / referencia  
    if "fecha_cliente" in work.columns:
        fechas = pd.to_datetime(work["fecha_cliente"], errors="coerce")

        # Intento adicional para formatos DD-MM-YYYY como en los datos de inferencia
        mask_na = fechas.isna() & work["fecha_cliente"].notna()
        if mask_na.any():
            fechas_alt = pd.to_datetime(
                work.loc[mask_na, "fecha_cliente"],
                errors="coerce",
                dayfirst=True,
            )
            fechas.loc[mask_na] = fechas_alt

        work["fecha_cliente"] = fechas

    # as_of_date (estable por archivo; evita usar "hoy" si hay fecha_cliente)
    default_as_of = pd.Timestamp("2025-10-27")
    if meta and meta.get("as_of_date"):
        as_of = pd.to_datetime(meta["as_of_date"])
    else:
        as_of = default_as_of
    # edad (si falta)  
    if "edad" not in work.columns:
        col_birth = _first_existing(work, ["anio_nacimiento", "Year_Birth", "year_birth", "birth_year"])
        if col_birth is not None:
            birth = pd.to_numeric(work[col_birth], errors="coerce")
            work["edad"] = (as_of.year - birth).astype("float")

    # anio_alta (si falta)  
    if "anio_alta" not in work.columns and "fecha_cliente" in work.columns:
        work["anio_alta"] = work["fecha_cliente"].dt.year

    # antiguedad (si falta)  
    if "fecha_cliente" in work.columns and work["fecha_cliente"].notna().any():
        if "antiguedad_dias" not in work.columns:
            work["antiguedad_dias"] = (as_of - work["fecha_cliente"]).dt.days.astype("float")
        if "antiguedad_anios" not in work.columns:
            work["antiguedad_anios"] = (work["antiguedad_dias"] / 365.25).astype("float")

    # estado_civil (01B): homogeneizar categorías EN->ES y validar  
    if "estado_civil" in work.columns:
        estados_validos = {"Casado", "Soltero", "Divorciado", "Union_Libre", "Viudo"}
        estado_map = {
            "Married": "Casado",
            "Together": "Union_Libre",
            "Single": "Soltero",
            "Divorced": "Divorciado",
            "Widow": "Viudo",
            "Alone": "Soltero",
        }
        if work["estado_civil"].dtype == object:
            est_raw = work["estado_civil"].astype(str).str.strip()
            est_raw = est_raw.replace({"nan": np.nan, "None": np.nan, "none": np.nan, "": np.nan})
            est_raw = est_raw.replace(estado_map)
            desconocidos = sorted(set(est_raw.dropna().unique()) - estados_validos)
            if desconocidos:
                print(f"Valores 'estado_civil' no mapeables (01B) -> se marcarán como NaN: {desconocidos}")
                est_raw = est_raw.where(~est_raw.isin(desconocidos))
            work["estado_civil"] = est_raw
    # tiene_pareja (01B) con soporte de valores EN  
    if "estado_civil" in work.columns and "tiene_pareja" not in work.columns:
        pareja_vals = {"Casado", "Union_Libre", "Married", "Together"}
        work["tiene_pareja"] = work["estado_civil"].isin(pareja_vals).astype(int)

    # educacion (01B) con soporte de valores EN + 'nan' string  
    if "educacion" in work.columns:
        educ_map = {
            "Basica": 1, "Secundaria": 2, "Universitaria": 3, "Master": 4, "Doctorado": 5,
            # equivalencias EN (dataset marketing campaign)
            "Basic": 1, "2n Cycle": 2, "Graduation": 3, "PhD": 5,
        }
        if work["educacion"].dtype == object:
            edu_raw = work["educacion"].astype(str).str.strip()
            edu_raw = edu_raw.replace({"nan": np.nan, "None": np.nan, "none": np.nan, "NULL": np.nan, "null": np.nan, "": np.nan})

            unk = sorted(set(edu_raw.dropna().unique()) - set(educ_map.keys()))
            if unk:
                raise ValueError(f"Valores 'educacion' no mapeables (01B): {unk}")

            work["educacion"] = edu_raw.map(educ_map)

    # hijos_casa (01B) con soporte EN  
    if "hijos_casa" in work.columns and "adolescentes_casa" in work.columns:
        work["hijos_casa"] = pd.to_numeric(work["hijos_casa"], errors="coerce").fillna(0) + pd.to_numeric(work["adolescentes_casa"], errors="coerce").fillna(0)
    elif "Kidhome" in work.columns and "Teenhome" in work.columns and "hijos_casa" not in work.columns:
        work["hijos_casa"] = pd.to_numeric(work["Kidhome"], errors="coerce").fillna(0) + pd.to_numeric(work["Teenhome"], errors="coerce").fillna(0)

    # educacion_x_estado (01B)  
    if {"educacion", "tiene_pareja"}.issubset(work.columns) and "educacion_x_estado" not in work.columns:
        work["educacion_x_estado"] = pd.to_numeric(work["educacion"], errors="coerce") * pd.to_numeric(work["tiene_pareja"], errors="coerce")

    # gastos: derivar gasto_total / props / categorias_compradas / gasto_promedio si faltan  
    gasto_candidates = {
        "gasto_vinos": ["gasto_vinos", "MntWines"],
        "gasto_frutas": ["gasto_frutas", "MntFruits"],
        "gasto_carnes": ["gasto_carnes", "MntMeatProducts"],
        "gasto_pescado": ["gasto_pescado", "MntFishProducts"],
        "gasto_dulces": ["gasto_dulces", "MntSweetProducts"],
        "gasto_oro": ["gasto_oro", "MntGoldProds"],
    }

    # normalizar nombres si vienen en EN y faltan en ES
    for es, cands in gasto_candidates.items():
        if es not in work.columns:
            src = _first_existing(work, cands)
            if src is not None:
                work[es] = pd.to_numeric(work[src], errors="coerce")

    gasto_cols = [c for c in gasto_candidates.keys() if c in work.columns]

    if "gasto_total" not in work.columns and gasto_cols:
        work["gasto_total"] = work[gasto_cols].fillna(0).sum(axis=1)

    if "categorias_compradas" not in work.columns and gasto_cols:
        work["categorias_compradas"] = (work[gasto_cols].fillna(0) > 0).sum(axis=1).astype(int)

    if "gasto_promedio" not in work.columns and "gasto_total" in work.columns:
        # elección mínima y consistente: promedio por categoría comprada (evita duplicar ticket_promedio)
        denom = work["categorias_compradas"] if "categorias_compradas" in work.columns else 0
        denom = pd.to_numeric(denom, errors="coerce").fillna(0)
        gt = pd.to_numeric(work["gasto_total"], errors="coerce").fillna(0)
        work["gasto_promedio"] = np.where(denom > 0, gt / denom, 0.0)

    for es, _ in gasto_candidates.items():
        prop = f"prop_{es}"
        if prop not in work.columns and es in work.columns and "gasto_total" in work.columns:
            num = pd.to_numeric(work[es], errors="coerce").fillna(0)
            den = pd.to_numeric(work["gasto_total"], errors="coerce").fillna(0)
            work[prop] = np.where(den > 0, num / den, 0.0)

    # compras: derivar compras_totales/offline/tasas/ticket si faltan  
    col_web = _first_existing(work, ["compras_web", "NumWebPurchases", "num_compras_web"])
    col_cat = _first_existing(work, ["compras_catalogo", "NumCatalogPurchases", "num_compras_catalogo"])
    col_store = _first_existing(work, ["compras_tienda", "NumStorePurchases", "num_compras_tienda"])
    col_deals = _first_existing(work, ["compras_oferta", "NumDealsPurchases", "num_compras_oferta"])

    def _to_numeric(col_name: str | None) -> pd.Series | None:
        if col_name and col_name in work.columns:
            return pd.to_numeric(work[col_name], errors="coerce").fillna(0)
        return None

    compras_web = _to_numeric(col_web)
    compras_cat = _to_numeric(col_cat)
    compras_store = _to_numeric(col_store)
    compras_deals = _to_numeric(col_deals)

    if "compras_online" in work.columns:
        work["compras_online"] = pd.to_numeric(work["compras_online"], errors="coerce").fillna(0)
    elif compras_web is not None and compras_cat is not None:
        work["compras_online"] = (compras_web + compras_cat).astype(float)

    if "compras_totales" not in work.columns:
        componentes = [s for s in [compras_web, compras_cat, compras_store, compras_deals] if s is not None]
        if componentes:
            total = componentes[0]
            for comp in componentes[1:]:
                total = total + comp
            work["compras_totales"] = total.astype(float)

    if "compras_offline" not in work.columns and compras_store is not None:
        work["compras_offline"] = compras_store.astype(float)

    if "ratio_compras_online" not in work.columns and "compras_online" in work.columns and "compras_totales" in work.columns:
        tot = pd.to_numeric(work["compras_totales"], errors="coerce").fillna(0)
        online = pd.to_numeric(work["compras_online"], errors="coerce").fillna(0)
        work["ratio_compras_online"] = np.where(tot > 0, online / tot, np.nan)

    if "tasa_compra_online" not in work.columns and "ratio_compras_online" in work.columns:
        work["tasa_compra_online"] = work["ratio_compras_online"]

    if "tasa_compra_oferta" not in work.columns and compras_deals is not None and "compras_totales" in work.columns:
        deals = compras_deals
        tot = pd.to_numeric(work["compras_totales"], errors="coerce").fillna(0)
        work["tasa_compra_oferta"] = np.where(tot > 0, deals / tot, 0.0)

    if "ticket_promedio" not in work.columns and "gasto_total" in work.columns and "compras_totales" in work.columns:
        gt = pd.to_numeric(work["gasto_total"], errors="coerce").fillna(0)
        tot = pd.to_numeric(work["compras_totales"], errors="coerce").fillna(0)
        work["ticket_promedio"] = np.where(tot > 0, gt / tot, 0.0)
    # hogar: tamano_hogar / dependientes / unipersonal si faltan  
    if "tamano_hogar" not in work.columns:
        if {"tiene_pareja", "hijos_casa"}.issubset(work.columns):
            work["tamano_hogar"] = (1 + pd.to_numeric(work["tiene_pareja"], errors="coerce").fillna(0) +
                                    pd.to_numeric(work["hijos_casa"], errors="coerce").fillna(0)).astype(float)
        elif "total_dependientes" in work.columns:
            work["tamano_hogar"] = (1 + pd.to_numeric(work["total_dependientes"], errors="coerce").fillna(0)).astype(float)

    if "tiene_dependientes" not in work.columns:
        if "hijos_casa" in work.columns:
            work["tiene_dependientes"] = (pd.to_numeric(work["hijos_casa"], errors="coerce").fillna(0) > 0).astype(int)
        elif "total_dependientes" in work.columns:
            work["tiene_dependientes"] = (pd.to_numeric(work["total_dependientes"], errors="coerce").fillna(0) > 0).astype(int)

    if "hogar_unipersonal" not in work.columns and "tamano_hogar" in work.columns:
        work["hogar_unipersonal"] = (pd.to_numeric(work["tamano_hogar"], errors="coerce").fillna(0) == 1).astype(int)

    # reglas determinísticas 01B (filtrado)  
    if "edad" in work.columns:
        work = work[work["edad"] <= 120].copy()
    if "ingresos" in work.columns:
        work = work[work["ingresos"] != 666666].copy()

    # ratio_compras_online: mismo criterio 01B (evita inf/NaN) -> dropna
    if "ratio_compras_online" in work.columns:
        work = work.dropna(subset=["ratio_compras_online"]).copy()

    # drop redundantes (01B)  
    cols_drop = ["total_dependientes", "compras_online", "adolescentes_casa", "anio_nacimiento", "fecha_cliente"]
    work = work.drop(columns=[c for c in cols_drop if c in work.columns])

    # normalizar columnas binarias tipo campaña (acepta_cmp*, reclama, etc.) a 0/1 como en 01B  
    if meta is not None:
        bin_cols_clf = meta.get("binary_cols", [])
        bin_cols_reg = meta.get("reg_binary_cols", [])
        bin_cols_all = set(bin_cols_clf) | set(bin_cols_reg)

        true_vals = {"yes", "si", "sí", "true", "1", "y", "t"}
        false_vals = {"no", "false", "0", "n", "f"}

        for col in bin_cols_all:
            if col in work.columns and work[col].dtype == object:
                s = work[col].astype(str).str.strip()
                s_lower = s.str.lower()
                mapped = np.where(
                    s_lower.isin(true_vals), 1,
                    np.where(s_lower.isin(false_vals), 0, np.nan)
                )
                # Intentar recuperar valores numéricos si vienen como strings "0"/"1"
                num_fallback = pd.to_numeric(s, errors="coerce")
                mapped = np.where(np.isnan(mapped), num_fallback, mapped)
                work[col] = pd.to_numeric(mapped, errors="coerce")

    # asegurar consistencia con entrenamiento: sin NaNs en features crudas usadas por el modelo  
    # En 01B el modelo se entrenó sin NaNs en las columnas de entrada; aquí replicamos ese supuesto
    if meta is not None:
        expected_raw = meta.get("raw_feature_names", [])
        needed = [c for c in expected_raw if c in work.columns]
        if needed:
            before = len(work)
            work = work.dropna(subset=needed).copy()
            after = len(work)
            if before != after:
                print(f"Filas descartadas por NaNs en features requeridas (01B): {before - after}")

    return work


def preparar_features(df: pd.DataFrame, meta: dict) -> Tuple[pd.DataFrame, pd.Series | None]:
    work = df.drop(columns=[TARGET_COL], errors="ignore").copy()
    expected = meta.get("raw_feature_names", [])
    work = validar_columnas(work, expected, "clasificacion (features crudas)", allow_extras=True)

    for col in work.columns:
        if str(work[col].dtype) == "Int64":
            work[col] = work[col].astype("int64")

    target = df[TARGET_COL] if TARGET_COL in df.columns else None
    return work, target


def preparar_regresion(df: pd.DataFrame, meta: dict) -> Tuple[pd.DataFrame, pd.Series | None]:
    work = df.drop(columns=[TARGET_COL], errors="ignore").copy()
    expected = meta.get("reg_raw_feature_names") or meta.get("raw_feature_names", [])
    work = validar_columnas(work, expected, "regresion (features crudas)", allow_extras=True)

    for col in work.columns:
        if str(work[col].dtype) == "Int64":
            work[col] = work[col].astype("int64")

    target = df[TARGET_COL] if TARGET_COL in df.columns else None
    return work, target


## Preparación del dataset para inferencia

Extraemos las features y la variable objetivo (si está disponible), validando que las columnas coincidan con las esperadas por el modelo.

In [208]:
# Aplicar el mismo preprocesamiento determinístico de 01B antes de alinear columnas
df_raw = preprocesamiento_deterministico_01b(df_raw, TARGET_COL, metadata)

# Preparar features según tarea (alineación exacta a metadata)
if TAREA == "clasificacion":
    X_inf, y_inf = preparar_features(df_raw, metadata)
elif TAREA == "regresion":
    X_inf, y_inf = preparar_regresion(df_raw, metadata)
else:
    raise ValueError(f"Tarea no reconocida: {TAREA}")

print(f"Features preparadas: {X_inf.shape[0]} observaciones, {X_inf.shape[1]} variables")
print(f"Target disponible: {y_inf is not None}")

resumen_dimensiones = pd.DataFrame({
    "observaciones": [X_inf.shape[0]],
    "features": [X_inf.shape[1]],
    "target_disponible": [y_inf is not None]
})
resumen_dimensiones


Valores 'estado_civil' no mapeables (01B) -> se marcarán como NaN: ['YOLO']
Filas descartadas por NaNs en features requeridas (01B): 5
Descartamos columnas extra en clasificacion (features crudas): ['id', 'nombre', 'apellidos', 'direccion', 'telefono1', 'telefono2', 'email', 'dni', 'tarjeta_credito_asociada', 'coste_contacto', 'ingresos_contacto']
Features preparadas: 324 observaciones, 47 variables
Target disponible: True


,observaciones,features,target_disponible
0,324,47,True


## Generación de predicciones

Aplicamos el pipeline entrenado sobre el dataset de inferencia.

In [209]:
if TAREA == "clasificacion":
    y_pred_proba = model.predict_proba(X_inf)[:, 1]
    clf_threshold = float(metadata.get("clf_threshold", 0.5))
    y_pred = (y_pred_proba >= clf_threshold).astype(int)

    print(f"Umbral aplicado: {clf_threshold:.4f}")
    distrib_pred = pd.Series(y_pred).value_counts().sort_index().to_frame("Frecuencia")
    distrib_pred["Porcentaje"] = (distrib_pred["Frecuencia"] / len(y_pred) * 100).round(2)
    distrib_pred.index = distrib_pred.index.map({0: "No Responde", 1: "Responde"})
    distrib_pred = distrib_pred.reindex(["No Responde", "Responde"], fill_value=0)
    distrib_pred
else:
    # El pipeline de 01B en regresión fue entrenado sobre log1p(y). La predicción sale en log-escala.
    y_pred_log = model.predict(X_inf)
    y_pred = np.expm1(y_pred_log)

    print("Estadísticas de las predicciones (EUROS):")
    resumen_pred = pd.DataFrame({"y_pred": pd.Series(y_pred).describe()})
    resumen_pred

Umbral aplicado: 0.3500


## Evaluación de métricas

Si el dataset incluye la variable objetivo, calculamos las métricas de rendimiento para evaluar la capacidad de generalización del modelo.

In [210]:
if y_inf is not None:
    if TAREA == "clasificacion":
        # (recalcular por seguridad)
        y_pred_proba = model.predict_proba(X_inf)[:, 1]
        clf_threshold = float(metadata.get("clf_threshold", 0.5))
        y_pred = (y_pred_proba >= clf_threshold).astype(int)

        metrics_clf = pd.DataFrame({
            "AUC": [roc_auc_score(y_inf, y_pred_proba)],
            "Accuracy": [accuracy_score(y_inf, y_pred)],
            "Precision": [precision_score(y_inf, y_pred, zero_division=0)],
            "Recall": [recall_score(y_inf, y_pred, zero_division=0)],
            "F1": [f1_score(y_inf, y_pred, zero_division=0)],
        }, index=["inferencia"])

        matriz_confusion = pd.DataFrame(
            confusion_matrix(y_inf, y_pred),
            index=["Real 0", "Real 1"],
            columns=["Pred 0", "Pred 1"],
        )
        display(metrics_clf)
        display(matriz_confusion)

        thresholds_grid = np.linspace(0.05, 0.95, 19)
        thresholds = np.unique(np.concatenate(([clf_threshold], thresholds_grid)))
        registros = []
        for thr in thresholds:
            y_tmp = (y_pred_proba >= thr).astype(int)
            registros.append({
                "threshold": thr,
                "Accuracy": accuracy_score(y_inf, y_tmp),
                "Precision": precision_score(y_inf, y_tmp, zero_division=0),
                "Recall": recall_score(y_inf, y_tmp, zero_division=0),
                "F1": f1_score(y_inf, y_tmp, zero_division=0),
            })

        df_thresholds = pd.DataFrame(registros).set_index("threshold")
        best_thr = df_thresholds["F1"].idxmax()
        print(f"Sugerencia: umbral que maximiza F1 en estos datos = {best_thr:.4f}")

        comparacion_dict = {"umbral_metadata": df_thresholds.loc[clf_threshold]}
        if abs(best_thr - clf_threshold) > 1e-9:
            comparacion_dict["umbral_sugerido"] = df_thresholds.loc[best_thr]
        comparacion = pd.DataFrame(comparacion_dict).T
        display(comparacion)

    else:
        # y_inf viene en euros (gasto_total). y_pred ya está en euros (expm1 aplicado).
        baseline_mae = (y_inf - y_inf.mean()).abs().mean()
        baseline_rmse = ((y_inf - y_inf.mean()) ** 2).mean() ** 0.5
        mae = mean_absolute_error(y_inf, y_pred)
        rmse = mean_squared_error(y_inf, y_pred) ** 0.5
        r2 = r2_score(y_inf, y_pred)

        metrics_reg = pd.DataFrame({
            "MAE": [mae],
            "RMSE": [rmse],
            "R2": [r2],
            "baseline_MAE": [baseline_mae],
            "baseline_RMSE": [baseline_rmse],
        }, index=["inferencia"])

        display(metrics_reg)
        distrib_pred = pd.DataFrame({
            "y_real": y_inf.describe(),
            "y_pred": pd.Series(y_pred).describe(),
        })
        distrib_pred

,AUC,Accuracy,Precision,Recall,F1
inferencia,0.942378,0.817901,0.465347,0.903846,0.614379


,Pred 0,Pred 1
Real 0,218,54
Real 1,5,47


Sugerencia: umbral que maximiza F1 en estos datos = 0.6000


,Accuracy,Precision,Recall,F1
umbral_metadata,0.817901,0.465347,0.903846,0.614379
umbral_sugerido,0.901235,0.661290,0.788462,0.719298


In [211]:
print("Inferencia completada.")
print(f"Tarea: {TAREA}")
print(f"Filas inferidas: {len(X_inf)}")


Inferencia completada.
Tarea: clasificacion
Filas inferidas: 324


## Síntesis

Cargamos los pipelines entrenados y aplicamos las predicciones sobre el dataset de inferencia. Si está disponible la variable objetivo, calculamos las métricas correspondientes para evaluar el rendimiento del modelo.

## Conclusiones

El notebook carga el pipeline entrenado y genera predicciones sobre el dataset de inferencia. Las métricas obtenidas permiten evaluar el desempeño del modelo sobre datos nuevos.